A quick notebook to show how distributed computing can be done very quickly in Julia. The following packages are used:

- `Distributed`
- `SharedArrays`

`Distributed` allows for the easy distribution of jobs, and `SharedArrays` is a required component to allow the distributed for loop to access the array from memory at all times.

In [15]:
# load the relevant packages
using Distributed
using SharedArrays

Set up the distributed computing cores

In [2]:
# count the number of CPU cores we have
n_cores = length(Sys.cpu_info())
# add cores to processes number
addprocs(n_cores - 1)
println("We have $(nprocs()) processes and $(nworkers()) workers")

We have 4 procs and 3 workers


Define some simple functions we want to test here

In [66]:
# simple function to create an array of cubed values
function get_i_cubed(i_start=1, i_end=100)
    i_cubed = Array{Int64}(undef, i_end)
    for i in i_start:i_end
        i_cubed[i] = i ^ 3
    end
    return i_cubed
end

# simple function to create an array of cubed values, 
# this time distributing amongst more cores
function get_i_cubed_dist(i_start=1, i_end=100)
    i_cubed = SharedArray{Int64}(i_end)
    @sync @distributed for i in i_start:i_end
        i_cubed[i] = i ^ 3
    end
    return i_cubed
end

get_i_cubed_dist (generic function with 3 methods)

In [70]:
@time cubed_array = get_i_cubed(1, 10^8)

  5.089936 seconds (19 allocations: 762.940 MiB, 64.11% gc time)


100000000-element Array{Int64,1}:
                   1
                   8
                  27
                  64
                 125
                 216
                 343
                 512
                 729
                1000
                1331
                1728
                2197
                   ⋮
 1673764241506895309
 1703764235206895640
 1733764229506895911
 1763764224406896128
 1793764219906896297
 1823764216006896424
 1853764212706896515
 1883764210006896576
 1913764207906896613
 1943764206406896632
 1973764205506896639
 2003764205206896640

In [71]:
@time cubed_array_dist = get_i_cubed_dist(1, 10^8)

  1.548732 seconds (845 allocations: 39.500 KiB)


100000000-element SharedArray{Int64,1}:
                   1
                   8
                  27
                  64
                 125
                 216
                 343
                 512
                 729
                1000
                1331
                1728
                2197
                   ⋮
 1673764241506895309
 1703764235206895640
 1733764229506895911
 1763764224406896128
 1793764219906896297
 1823764216006896424
 1853764212706896515
 1883764210006896576
 1913764207906896613
 1943764206406896632
 1973764205506896639
 2003764205206896640